# Prediction & Evaluation on Unlabel

This notebook uses the pretrained encoder and fusion model to:

1. Extract a PreCNN feature vector $z_{\text{pre}}$ from amplitude windows.
2. Extract a physics feature $z_{\text{phys}}$ from phase / AoA.
3. For each CSI window, predict:
   - presence probabilities $p = (p_{\text{empty}}, p_{\text{nonempty}})$,
   - location coordinates $\hat{\mathbf{p}} = (\hat{x}, \hat{y})$.
4. Save all predictions and plot the predicted locations for each unlabeled set.

## Imports & Paths

In [8]:
import os
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

print("TensorFlow:", tf.__version__)

# --- TRAIN (for mu/sigma & model paths) ---
TRAIN_ROOT      = "/home/tonyliao/Location_new_aoa_PDF"
ENC_SAVE_PATH   = os.path.join(TRAIN_ROOT, "models/prec_encoder.h5")
FINAL_MODEL     = os.path.join(TRAIN_ROOT, "models/final_finetuned_model.h5")

# --- UNLABELED DATA ROOT(S) ---
# change these to your unlabeled folders, e.g. Top_Unlabel, Middle_Unlabel...
UNLABEL_ROOTS = [
    "/home/tonyliao/Location_new_aoa_PDF/RuJun",
    "/home/tonyliao/Location_new_aoa_PDF/Top_Unlab",
    "/home/tonyliao/Location_new_aoa_PDF/YuXiang",
    "/home/tonyliao/Location_new_aoa_PDF/Empty_Unlab",
    "/home/tonyliao/Location_new_aoa_PDF/BoYu",
    "/home/tonyliao/Location_new_aoa_PDF/YuanQi",
    "/home/tonyliao/Location_new_aoa_PDF/Jody",
    "/home/tonyliao/Location_new_aoa_PDF/Door_5mins_Active",
    "/home/tonyliao/Location_new_aoa_PDF/Door_5mins_Passive",
    "/home/tonyliao/Location_new_aoa_PDF/YuXiang_5mins_Active",
    "/home/tonyliao/Location_new_aoa_PDF/YuXiang_5mins_Passive",
    "/home/tonyliao/Location_new_aoa_PDF/Bottom_5mins_Active",
    "/home/tonyliao/Location_new_aoa_PDF/Bottom_5mins_Passive",
]

# Physics config (same as Step 3)
USE_AOA = True
C   = 3e8
FC  = 5.18e9
LAM = C / FC
D   = 0.05
TAU_AOA = 0.6
aoa_confs = []
aoa_oks   = []

coord_mean = np.load("/home/tonyliao/Location_new_aoa_PDF/models/coord_mean.npy")
coord_std  = np.load("/home/tonyliao/Location_new_aoa_PDF/models/coord_std.npy")
BIAS_PATH = "/home/tonyliao/Location_new_aoa_PDF/models/coord_bias.npy"

if os.path.exists(BIAS_PATH):
    coord_bias = np.load(BIAS_PATH).astype(np.float32)
else:
    coord_bias = np.zeros((2,), dtype=np.float32)

print("coord_bias:", coord_bias, "| from:", BIAS_PATH)

TensorFlow: 2.19.0
coord_bias: [0.00610671 1.6157156 ] | from: /home/tonyliao/Location_new_aoa_PDF/models/coord_bias.npy


## Load Encoder & Final Model

The pretrained encoder maps each window to a 128-D embedding:

$$
z_{\text{pre}} \in \mathbb{R}^{128}.
$$

The fusion model takes $(z_{\text{pre}}, z_{\text{phys}})$ and outputs:

- a presence softmax
  $$
  p_k = \Pr(y = k \mid z_{\text{pre}}, z_{\text{phys}}),
  \qquad
  \sum_{k \in \{\text{empty}, \text{nonempty}\}} p_k = 1,
  $$
- and regressed coordinates
  $$
  \hat{\mathbf{p}} = (\hat{x}, \hat{y}) \in \mathbb{R}^2.
  $$

The predicted class is

$$
\hat{y} = \arg\max_k p_k.
$$

In [9]:
# Load pretrained encoder & final multi-task model (no training here)
encoder    = tf.keras.models.load_model(ENC_SAVE_PATH)
full_model = tf.keras.models.load_model(FINAL_MODEL)
encoder.summary()
full_model.summary()
DIM_PRE  = encoder.output_shape[-1]
print("DIM_PRE:", DIM_PRE)

Model: "PreCNN_Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4, 159)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 4, 64)          │        50,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 2, 128)         │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 2, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 1, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ precnn_embedding (Dense)        │ (None, 128)            │        32,896 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,280 (880.00 KB)

 Trainable params: 224,384 (876.50 KB)

 Non-trainable params: 896 (3.50 KB)

...

 Total params: 51,601 (201.57 KB)

 Trainable params: 51,599 (201.56 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

DIM_PRE: 128


In [10]:
# ---------------------------------------------------
# Load PreCNN *classifier* model for presence gating
# ---------------------------------------------------
PRECNN_CLF_PATH = "/home/tonyliao/Location_new_aoa_PDF/models/prec_classifier.h5"  # <-- update to your actual clf path

precnn_clf = tf.keras.models.load_model(PRECNN_CLF_PATH)
precnn_clf.summary()

# class mapping assumption: index 0 = EMPTY, index 1 = NON-EMPTY
P_NONEMPTY_THR = 0.6  # threshold; you can tune (0.5–0.7 typical)

Model: "PreCNN_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4, 159)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 4, 64)          │        50,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 2, 128)         │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 2, 128)         │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 1, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ precnn_embedding (Dense)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 226,572 (885.05 KB)

 Trainable params: 225,674 (881.54 KB)

 Non-trainable params: 896 (3.50 KB)

 Optimizer params: 2 (12.00 B)

## Shared Helpers (Amplitude → PreCNN Input)

From MATLAB we have amplitude windows

$$
H(t,s,a) \in \mathbb{R}^{T \times S \times A},
$$

where $t$ is time index, $s$ is subcarrier and $a$ is RX antenna.

We average over antennas:

$$
\tilde{H}(t,s) = \frac{1}{A} \sum_{a=1}^{A} H(t,s,a),
$$

transpose to channel-first:

$$
X_{\text{amp}} = \tilde{H}^{\top} \in \mathbb{R}^{S \times T},
$$

and normalize each subcarrier channel using global statistics $(\mu_c, \sigma_c)$
computed from the training set:

$$
X_{\text{norm}}(c,t) = \frac{X_{\text{amp}}(c,t) - \mu_c}{\sigma_c + 10^{-6}}.
$$

Then we append temporal mean and std per channel to build the
PreCNN input $X_{\text{pre}}(t,:)$ (same as in the training notebook).

In [11]:
def list_amp_files(root):
    return sorted(glob.glob(os.path.join(root, "**", "amp_window_*.npy"),
                            recursive=True))

def amp_to_pha_path(amp_path):
    return amp_path.replace("amp_window_", "pha_window_")


# --- compute global mu/sigma from TRAIN_ROOT (same style as Step 2) ---

def compute_mu_sigma(paths, sample_limit=None):
    sums    = None
    sq_sums = None
    count   = 0

    for i, p in enumerate(paths):
        if sample_limit and i >= sample_limit:
            break

        raw         = np.load(p)               # [T, S, A]
        avecsi_like = raw.mean(axis=2)         # [T, S]
        amp_ct      = avecsi_like.T            # [S, T]

        C_base, _ = amp_ct.shape
        if sums is None:
            sums    = np.zeros((C_base,), dtype=np.float64)
            sq_sums = np.zeros((C_base,), dtype=np.float64)

        sums    += amp_ct.mean(axis=1)
        sq_sums += (amp_ct**2).mean(axis=1)
        count   += 1

    mu    = sums / count
    sigma = np.sqrt(sq_sums / count - mu**2)
    return mu.astype(np.float32), sigma.astype(np.float32)


print("Computing mu/sigma from TRAIN_ROOT ...")
train_amp_files = list_amp_files(TRAIN_ROOT)
mu_global, sigma_global = compute_mu_sigma(train_amp_files)
print("mu_global:", mu_global.shape, "sigma_global:", sigma_global.shape)


# --- PreCNN stat channels (same as Step 2) ---

USE_PRECNN_STATS = True

def add_precnn_stats_channels(amp_ct):
    C_base, T = amp_ct.shape
    mu = amp_ct.mean(axis=1, keepdims=True)
    sd = amp_ct.std(axis=1, keepdims=True) + 1e-6

    mu_rep = np.repeat(mu, T, axis=1)
    sd_rep = np.repeat(sd, T, axis=1)

    return np.concatenate([amp_ct, mu_rep, sd_rep], axis=0).astype(np.float32)


def load_precnn_input(path, mu=None, sigma=None):
    """
    path: amp_window_XXXXX.npy, shape [T, S, A]
    output: X_pre [T, C_pre] for encoder
    """
    amp = np.load(path).astype(np.float32)  # [T, S, A]

    # average antennas
    avecsi_like = amp.mean(axis=2)          # [T, S]

    # [S, T]
    amp_ct = avecsi_like.T                  # [C_base, T]

    # normalize using global mu/sigma from TRAIN_ROOT
    if (mu is not None) and (sigma is not None):
        amp_ct = (amp_ct - mu[:, None]) / (sigma[:, None] + 1e-6)

    if USE_PRECNN_STATS:
        amp_ct = add_precnn_stats_channels(amp_ct)  # [C_pre, T]

    X_pre = np.transpose(amp_ct, (1, 0))            # [T, C_pre]
    return X_pre

Computing mu/sigma from TRAIN_ROOT ...
mu_global: (53,) sigma_global: (53,)


## Phase / AoA Helpers (Physics Branch, Step 3)

Let the phase window be

$$
\phi(t,s,a) = \arg H(t,s,a),
$$

with shape $[T, S, A]$.

### Phase detrending and unwrapping

For each time $t$ and antenna $a$, we fit a line over subcarriers

$$
\phi(t,s,a) \approx k_{t,a} \, s + b_{t,a},
$$

and subtract it:

$$
\tilde{\phi}(t,s,a) = \phi(t,s,a) - \big(k_{t,a} \, s + b_{t,a}\big),
$$

then unwrap along the subcarrier axis.

### AoA feature from phase difference

For antennas $a = 1,2$ we form the unwrapped phase difference

$$
\Delta\phi(t,s) = \tilde{\phi}(t,s,2) - \tilde{\phi}(t,s,1).
$$

We average over time and subcarriers:

$$
\overline{\Delta\phi}
= \frac{1}{T S} \sum_{t=1}^{T} \sum_{s=1}^{S} \Delta\phi(t,s).
$$

With wavelength $\lambda = c / f_c$ and antenna spacing $D$, a ULA gives

$$
\sin\theta =
\operatorname{clip}
\left(
\frac{\lambda \, \overline{\Delta\phi}}{2 \pi D},
-1, 1
\right),
\qquad
\theta = \arcsin(\sin\theta).
$$

The physics feature used by the model is

$$
z_{\text{phys}} =
\begin{bmatrix}
\sin\theta \\
\cos\theta
\end{bmatrix}
\in \mathbb{R}^{2}.
$$

If AoA is not used or only one antenna is available, we instead build a
phase-pattern feature from antenna 1:

$$
\mu_s = \frac{1}{T} \sum_{t=1}^{T} \tilde{\phi}(t,s,1),
\qquad
\sigma_s = \sqrt{
\frac{1}{T} \sum_{t=1}^{T} \tilde{\phi}(t,s,1)^2 - \mu_s^2
},
$$

optionally downsampled over subcarriers and concatenated into a vector
$z_{\text{phys}}$.

In [12]:
DIM_PHYS = 5  # <- expects 5

def _pad_or_trim(x, dim=DIM_PHYS):
    x = np.asarray(x, dtype=np.float32).reshape(-1)
    if x.size >= dim:
        return x[:dim]
    return np.pad(x, (0, dim - x.size), mode="constant")

def detrend_phase(phi_ts):
    """
    phi_ts: [T, S] radians
    remove linear trend over subcarriers for each t
    """
    T, S = phi_ts.shape
    s_idx = np.arange(S)
    out = np.zeros_like(phi_ts, dtype=np.float32)

    for t in range(T):
        k, b = np.polyfit(s_idx, phi_ts[t], deg=1)
        out[t] = phi_ts[t] - (k * s_idx + b)

    return out


def compute_clean_phase_from_file(pha_path):
    """
    pha_path: pha_window_XXXXX.npy
    shape: [T, S, A]
    returns list of [T, S] per antenna (detrend + unwrap)
    """
    phi_raw = np.load(pha_path).astype(np.float32)   # [T, S, A]
    T, S, A = phi_raw.shape

    phi_clean = []
    for a in range(A):
        phi = phi_raw[:, :, a]              # [T, S]
        phi_dt  = detrend_phase(phi)
        phi_unw = np.unwrap(phi_dt, axis=1)
        phi_clean.append(phi_unw)

    return phi_clean


def compute_aoa_feature(phi_clean):
    """
    phi_clean: list of [T, S], uses first 2 antennas
    returns: [sin(theta), cos(theta)]
    """
    if len(phi_clean) < 2:
        raise ValueError("AoA mode requires ≥2 RX antennas")

    phi1 = phi_clean[0]
    phi2 = phi_clean[1]

    dphi     = phi2 - phi1
    dphi_unw = np.unwrap(dphi, axis=1)
    dphi_bar = np.mean(dphi_unw)

    arg = (LAM * dphi_bar) / (2.0 * np.pi * D)
    arg = np.clip(arg, -1.0, 1.0)
    theta = np.arcsin(arg)

    return np.array([np.sin(theta), np.cos(theta)], dtype=np.float32)

def compute_aoa_feature_and_conf(phi_clean):
    if len(phi_clean) < 2:
        raise ValueError("AoA mode requires ≥2 RX antennas")

    phi1 = phi_clean[0]
    phi2 = phi_clean[1]

    dphi     = phi2 - phi1
    dphi_unw = np.unwrap(dphi, axis=1)

    # stats
    dphi_bar = float(np.mean(dphi_unw))
    dphi_std = float(np.std(dphi_unw))

    # confidence: smaller variance => more stable AoA
    aoa_conf = float(1.0 / (dphi_std + 1e-6))

    # AoA
    arg = (LAM * dphi_bar) / (2.0 * np.pi * D)
    arg = np.clip(arg, -1.0, 1.0)
    theta = float(np.arcsin(arg))

    # 5-D feature (matches model input)
    # [sinθ, cosθ, conf, dphi_bar, dphi_std]
    z_phys = np.array([np.sin(theta), np.cos(theta), aoa_conf, dphi_bar, dphi_std], dtype=np.float32)
    return _pad_or_trim(z_phys), aoa_conf


def compute_phase_pattern(phi_clean, step=4):
    """
    Fallback if AoA not used or only 1 antenna.
    Build a feature then pad/trim to DIM_PHYS.
    """
    phi = phi_clean[0]                # [T, S]
    mu_s = np.mean(phi, axis=0)       # [S]
    sd_s = np.std(phi,  axis=0)       # [S]

    mu_ds = mu_s[::step]
    sd_ds = sd_s[::step]

    feat = np.concatenate([mu_ds, sd_ds]).astype(np.float32)
    return _pad_or_trim(feat)


def step3_phase_branch(pha_path):
    phi_clean = compute_clean_phase_from_file(pha_path)

    if USE_AOA and len(phi_clean) >= 2:
        z_phys, aoa_conf = compute_aoa_feature_and_conf(phi_clean)  # (5,)
        aoa_ok = True
    else:
        z_phys = compute_phase_pattern(phi_clean)                   # (5,)
        aoa_conf = 0.0
        aoa_ok = False

    # final safety
    z_phys = _pad_or_trim(z_phys)
    return z_phys, float(aoa_conf), bool(aoa_ok)

## Helper to Extract $(z_{\text{pre}}, z_{\text{phys}})$ and Predict

For each unlabeled window:

1. Build normalized amplitude input $X_{\text{pre}}$ and encode it to
   $z_{\text{pre}} \in \mathbb{R}^{128}$ using the pretrained encoder.
2. Build physics feature $z_{\text{phys}}$ from phase (AoA or phase-pattern).
3. Feed $(z_{\text{pre}}, z_{\text{phys}})$ into the fusion model to obtain
   presence probabilities $p$ and coordinates $\hat{\mathbf{p}}$.

In [13]:
def get_z_pre_from_amp(amp_path):
    X_pre = load_precnn_input(amp_path, mu_global, sigma_global)   # [T, C_pre]
    X_pre_batch = X_pre[np.newaxis, ...]                           # [1, T, C_pre]
    z_pre = encoder.predict(X_pre_batch, verbose=0)[0]             # [DIM_PRE]
    return z_pre.astype(np.float32)

def predict_one_window(amp_path):
    pha_path = amp_to_pha_path(amp_path)
    if not os.path.exists(pha_path):
        raise FileNotFoundError(f"Missing phase file: {pha_path}")

    # 1) Presence from PreCNN classifier (amplitude only)
    probs_presence, cls_presence = get_presence_from_precnn(amp_path)

    # 2) Features for localization model
    z_pre  = get_z_pre_from_amp(amp_path)            # [DIM_PRE]
    z_phys, aoa_conf, aoa_ok = step3_phase_branch(pha_path)# ([DIM_PHYS], conf)

    z_pre_in  = z_pre[np.newaxis, :]
    z_phys_in = z_phys[np.newaxis, :]

    # 3) Full model inference (presence head + coords head)
    out = full_model.predict([z_pre_in, z_phys_in], verbose=0)

    if isinstance(out, (list, tuple)):
        _presence_out, coords_n = out
    else:
        _presence_out = out["presence"]
        coords_n      = out["coords"]

    coords_n = coords_n[0]  # normalized coords if you trained normalized

    # De-normalize to meters (ONLY if model trained on normalized coords)
    coords = coords_n * coord_std + coord_mean

    # Use PreCNN as your final presence decision
    cls   = cls_presence
    probs = probs_presence

    aoa_ok = bool(aoa_conf >= TAU_AOA)

    return cls, probs, coords, z_pre, z_phys, aoa_conf, aoa_ok



def get_presence_from_precnn(amp_path):
    """
    Use the PreCNN classifier to get presence probabilities.

    Returns:
        probs_presence: [2] vector [p_empty, p_nonempty]
        cls_presence : 0 (EMPTY) or 1 (NON-EMPTY)
    """
    # reuse same preprocessing as encoder
    X_pre = load_precnn_input(amp_path, mu_global, sigma_global)  # [T, C_pre]
    X_pre_batch = X_pre[np.newaxis, ...]                          # [1, T, C_pre]

    probs = precnn_clf.predict(X_pre_batch, verbose=0)[0]         # [2]
    cls   = int(np.argmax(probs))                                 # 0=EMPTY,1=NON-EMPTY
    return probs.astype(np.float32), cls

## Run Prediction on Unlabeled Folders (No Training)

In [14]:
MAX_SAMPLES = None  # set int for speed, or None for all

RESULTS = {}  # folder → dict of arrays

for root in UNLABEL_ROOTS:
    amp_files = list_amp_files(root)
    total_windows = len(amp_files)

    print(f"\n=== Folder: {root} ===")
    print(f"Total windows found: {total_windows}")

    # ---------------------------------------
    # Random selection of MAX_SAMPLES windows
    # ---------------------------------------
    if MAX_SAMPLES is not None and total_windows > MAX_SAMPLES:
        idx = np.random.choice(total_windows, MAX_SAMPLES, replace=False)
        amp_files = [amp_files[i] for i in idx]
    else:
        if MAX_SAMPLES is not None:
            print(f"Total windows less than MAX_SAMPLES ({MAX_SAMPLES}), using all.")

    print(f"Windows selected for prediction: {len(amp_files)}")

    preds_cls       = []
    preds_probs     = []
    preds_coords    = []
    feats_pre_list  = []
    feats_phys_list = []
    valid_files     = []

    # IMPORTANT: reset per-folder
    aoa_confs = []
    aoa_oks   = []

    # ---------------------------------------
    # Prediction loop
    # ---------------------------------------
    for i, amp_p in enumerate(amp_files):
        try:
            cls, probs, coords, z_pre, z_phys, aoa_conf, aoa_ok = predict_one_window(amp_p)
            coords = coords - coord_bias
        except FileNotFoundError as e:
            print("WARN:", e)
            continue

        # Optional: skip EMPTY predictions entirely
        if cls == 0:
            continue

        preds_cls.append(cls)
        preds_probs.append(probs)
        preds_coords.append(coords)
        feats_pre_list.append(z_pre)
        feats_phys_list.append(z_phys)
        valid_files.append(amp_p)

        aoa_confs.append(aoa_conf)
        aoa_oks.append(aoa_ok)

        if (i + 1) % 2000 == 0:
            print(f"  processed {i+1}/{len(amp_files)}")

    if not valid_files:
        print("No valid window pairs in this folder.")
        continue

    # ---------------------------------------
    # Stack arrays
    # ---------------------------------------
    preds_cls    = np.array(preds_cls, dtype=np.int32)
    preds_probs  = np.vstack(preds_probs)           # [N,2]
    preds_coords = np.vstack(preds_coords)          # [N,2]
    Z_PRE_unlab  = np.vstack(feats_pre_list)        # [N,DIM_PRE]
    Z_PHYS_unlab = np.vstack(feats_phys_list)       # [N,DIM_PHYS]
    valid_files  = np.array(valid_files)

    aoa_confs = np.array(aoa_confs, dtype=np.float32)  # [N]
    aoa_oks   = np.array(aoa_oks, dtype=bool)          # [N]

    # ---------------------------------------
    # AoA-gated median report
    # ---------------------------------------
    coords_all = preds_coords
    coords_ok  = preds_coords[aoa_oks]

    print("Median (all windows):", np.median(coords_all, axis=0))

    if coords_ok.shape[0] >= 20:
        print(f"Median (AoA-gated, conf>={TAU_AOA}, N={coords_ok.shape[0]}):",
              np.median(coords_ok, axis=0))
    else:
        print(f"Too few AoA-stable windows (N={coords_ok.shape[0]}), fallback to all.")

    # ---------------------------------------
    # Store in RESULTS
    # ---------------------------------------
    RESULTS[root] = {
        "files":    valid_files,
        "cls":      preds_cls,
        "probs":    preds_probs,
        "coords":   preds_coords,
        "Z_PRE":    Z_PRE_unlab,
        "Z_PHYS":   Z_PHYS_unlab,
        "AOA_CONF": aoa_confs,
        "AOA_OK":   aoa_oks.astype(np.int32),
    }

    # ---------------------------------------
    # Summary
    # ---------------------------------------
    N = preds_cls.shape[0]
    n_empty  = np.sum(preds_cls == 0)   # should be 0 if you skip cls==0
    n_nonemp = np.sum(preds_cls == 1)

    print(f"Valid windows: {N}")
    print(f"Predicted EMPTY    : {n_empty} ({100.0*n_empty/N:.2f}%)")
    print(f"Predicted NON-EMPTY: {n_nonemp} ({100.0*n_nonemp/N:.2f}%)")



=== Folder: /home/tonyliao/Location_new_aoa_PDF/RuJun ===
Total windows found: 18453
Windows selected for prediction: 18453


  processed 2000/18453
  processed 4000/18453
  processed 6000/18453
  processed 8000/18453
  processed 10000/18453
  processed 12000/18453
  processed 14000/18453
  processed 16000/18453
  processed 18000/18453
Median (all windows): [1.4379289 0.4027773]
Median (AoA-gated, conf>=0.6, N=5588): [1.7456565  0.39159656]
Valid windows: 18453
Predicted EMPTY    : 0 (0.00%)
Predicted NON-EMPTY: 18453 (100.00%)

=== Folder: /home/tonyliao/Location_new_aoa_PDF/Top_Unlab ===
Total windows found: 18079
Windows selected for prediction: 18079
  processed 2000/18079
  processed 4000/18079
  processed 6000/18079
  processed 8000/18079
  processed 10000/18079
  processed 12000/18079
  processed 14000/18079
  processed 16000/18079
  processed 18000/18079
Median (all windows): [2.0103426 2.4078574]
Median (AoA-gated, conf>=0.6, N=12518): [2.0100431 2.407396 ]
Valid windows: 18079
Predicted EMPTY    : 0 (0.00%)
Predicted NON-EMPTY: 18079 (100.00%)

=== Folder: /home/tonyliao/Location_new_aoa_PDF/YuXiang

## Save Predictions for Later Analysis

In [15]:
import os, numpy as np
from datetime import datetime

TODAY = datetime.now().strftime("%Y%m%d")

# MUST match what you used in the prediction loop cell
# (set this ONCE at the top of the notebook and reuse everywhere)
# MAX_SAMPLES = None  # or 500, etc.

SUFFIX  = f"_{TODAY}_{MAX_SAMPLES}"
OUT_DIR = f"/home/tonyliao/Location_new_aoa_PDF/unlabel_preds_{SUFFIX}"
os.makedirs(OUT_DIR, exist_ok=True)

for root, data in RESULTS.items():
    name = os.path.basename(root.rstrip("/"))
    np.save(os.path.join(OUT_DIR, f"{name}_files.npy"),    data["files"])
    np.save(os.path.join(OUT_DIR, f"{name}_cls.npy"),      data["cls"])
    np.save(os.path.join(OUT_DIR, f"{name}_probs.npy"),    data["probs"])
    np.save(os.path.join(OUT_DIR, f"{name}_coords.npy"),   data["coords"])
    np.save(os.path.join(OUT_DIR, f"{name}_Z_PRE.npy"),    data["Z_PRE"])
    np.save(os.path.join(OUT_DIR, f"{name}_Z_PHYS.npy"),   data["Z_PHYS"])
    np.save(os.path.join(OUT_DIR, f"{name}_aoa_conf.npy"), data["AOA_CONF"])
    np.save(os.path.join(OUT_DIR, f"{name}_aoa_ok.npy"),   data["AOA_OK"])
    print(f"Saved predictions for {name} → {OUT_DIR}")


Saved predictions for RuJun → /home/tonyliao/Location_new_aoa_PDF/unlabel_preds__20260115_None
Saved predictions for Top_Unlab → /home/tonyliao/Location_new_aoa_PDF/unlabel_preds__20260115_None
Saved predictions for YuXiang → /home/tonyliao/Location_new_aoa_PDF/unlabel_preds__20260115_None
Saved predictions for Empty_Unlab → /home/tonyliao/Location_new_aoa_PDF/unlabel_preds__20260115_None
Saved predictions for BoYu → /home/tonyliao/Location_new_aoa_PDF/unlabel_preds__20260115_None
Saved predictions for YuanQi → /home/tonyliao/Location_new_aoa_PDF/unlabel_preds__20260115_None
Saved predictions for Jody → /home/tonyliao/Location_new_aoa_PDF/unlabel_preds__20260115_None
Saved predictions for Door_5mins_Active → /home/tonyliao/Location_new_aoa_PDF/unlabel_preds__20260115_None
Saved predictions for YuXiang_5mins_Active → /home/tonyliao/Location_new_aoa_PDF/unlabel_preds__20260115_None
Saved predictions for YuXiang_5mins_Passive → /home/tonyliao/Location_new_aoa_PDF/unlabel_preds__20260115_N

## Plotting Location Predictions and Median Error

For one unlabeled set we have predicted coordinates
$\hat{\mathbf{p}}_i = (\hat{x}_i, \hat{y}_i)$ for $i = 1, \dots, N$
and a chosen ground-truth location
$\mathbf{p}^\star = (x^\star, y^\star)$.

We report:

- the median prediction
  $$
  \tilde{\mathbf{p}} =
  \big(
  \operatorname{median}_i \hat{x}_i,\;
  \operatorname{median}_i \hat{y}_i
  \big),
  $$
- and the Euclidean error of this median point
  $$
  e_{\text{median}} =
  \left\|
  \tilde{\mathbf{p}} - \mathbf{p}^\star
  \right\|_2
  =
  \sqrt{
    (\tilde{x} - x^\star)^2 +
    (\tilde{y} - y^\star)^2
  }.
  $$

In [16]:
# ==============================================================
# Step: Plot Location Predictions for Unlabeled Sets + Metrics
# ==============================================================

import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

TODAY = datetime.now().strftime("%Y%m%d")
SUFFIX = f"_{TODAY}_{MAX_SAMPLES}"

# --------------------------------------------------------------
# Ground-truth (folder-level)
# --------------------------------------------------------------
GROUND_TRUTH = {
    "Top_Unlab":    (5, 0),
    "RuJun":        (2, 4),
    "YuXiang":      (3, 0),
    "BoYu":         (4, 0),
    "YuanQi":       (1, 0),
    "Jody":         (3, 4),
    "YuXiang_5mins_Active": (3,0),
    "YuXiang_5mins_Passive": (3,0),
    "Door_5mins_Active": (4,4),
    "Door_5mins_Passive": (4,4),
    "Bottom_5mins_Active": (1,2),
    "Bottom_5mins_Passive": (1,2),
    "Empty_Unlab":  None,
}

# Presence GT (folder-level only)
# 0 = Empty, 1 = Non-empty
PRESENCE_GT = {
    "Empty_Unlab": 0,
    "YuXiang_5mins_Active": 1,
    "YuXiang_5mins_Passive": 1,
    "Door_5mins_Active": 1,
    "Door_5mins_Passive": 1,
    "Bottom_5mins_Active": 1,
    "Bottom_5mins_Passive": 1,
    "Top_Unlab":   1,
    "RuJun":       1,
    "YuXiang":     1,
    "BoYu":        1,
    "YuanQi":      1,
    "Jody":        1,
}

# --------------------------------------------------------------
# Paths
# --------------------------------------------------------------
PRED_ROOT = f"/home/tonyliao/Location_new_aoa_PDF/unlabel_preds_{SUFFIX}"
FIG_ROOT  = f"/home/tonyliao/Location_new_aoa_PDF/unlabel_figures_{SUFFIX}"
os.makedirs(FIG_ROOT, exist_ok=True)

# --------------------------------------------------------------
# Helper: localization metrics
# --------------------------------------------------------------
def loc_metrics(coords, gt_xy):
    gt = np.array(gt_xy, dtype=np.float32).reshape(1, 2)
    err = np.linalg.norm(coords - gt, axis=1)

    out = {
        "N": len(err),
        "median_m": np.median(err),
        "mean_m": np.mean(err),
        "rmse_m": np.sqrt(np.mean(err**2)),
        "p90_m": np.percentile(err, 90),
    }
    for r in [0.5, 1.0, 2.0]:
        out[f"within_{r}m_%"] = (err <= r).mean() * 100.0

    med_xy = np.median(coords, axis=0)
    out["median_xy"] = med_xy
    out["median_xy_err_m"] = np.linalg.norm(med_xy - gt.reshape(2))
    return out

# --------------------------------------------------------------
# Helper: presence metrics (folder-level assumption)
# --------------------------------------------------------------
def presence_metrics_from_probs(probs, y_true, thr=0.5):
    # probs: Nx2 (softmax) or Nx1
    if probs.ndim == 2:
        p_nonempty = probs[:, 1]
    else:
        p_nonempty = probs.reshape(-1)

    y_pred = (p_nonempty >= thr).astype(int)

    acc = accuracy_score(y_true, y_pred)
    pr, rc, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="binary", pos_label=1, zero_division=0
    )
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    return acc, pr, rc, f1, cm

# --------------------------------------------------------------
# Plotting function
# --------------------------------------------------------------
def plot_coords(folder_name, coords, probs=None):
    gt = GROUND_TRUTH[folder_name]

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(coords[:, 0], coords[:, 1],
               s=10, alpha=0.35, color="blue",
               label="Predicted Points")

    text_lines = []

    # ---------------- Localization ----------------
    if gt is not None:
        m = loc_metrics(coords, gt)

        ax.scatter(gt[0], gt[1],
                   s=200, color="red", marker="*",
                   label="True Location")
        
        text_lines += [
            f"Median Error: {m['median_m']:.2f} m",
            # f"Mean Err: {m['mean_m']:.2f} m",
            # f"RMSE: {m['rmse_m']:.2f} m",
            # f"P90: {m['p90_m']:.2f} m",
            # f"≤1m: {m['within_1.0m_%']:.1f}%",
            f"Median XY: ({m['median_xy'][0]:.2f}, {m['median_xy'][1]:.2f})",
        ]

        print(f"[{folder_name}] Localization:")
        for k, v in m.items():
            if isinstance(v, float):
                print(f"  {k}: {v:.3f}")

    else:
        med_xy = np.median(coords, axis=0)
        text_lines += [
            "No GT location (EMPTY)",
            f"Median XY: ({med_xy[0]:.2f}, {med_xy[1]:.2f})",
        ]
        print(f"[{folder_name}] EMPTY — no localization GT")

    # ---------------- Presence (optional) ----------------
    if probs is not None and folder_name in PRESENCE_GT:
        y_true = np.full((probs.shape[0],), PRESENCE_GT[folder_name], dtype=int)
        acc, pr, rc, f1, cm = presence_metrics_from_probs(probs, y_true)

        text_lines += [
            "",
            f"Presence Accuracy: {acc:.2f}",
            f"Precision / Recall / F1:",
            f"{pr:.2f} / {rc:.2f} / {f1:.2f}",
        ]

        print(f"[{folder_name}] Presence:")
        print(f"  Accuracy={acc:.3f}  P={pr:.3f}  R={rc:.3f}  F1={f1:.3f}")
        print("  CM [[TN FP],[FN TP]]:\n", cm)

    # ---------------- Plot cosmetics ----------------
    ax.set_title(f"Predicted Locations — {folder_name}")
    ax.set_xlabel("X (m)")
    ax.set_ylabel("Y (m)")
    ax.grid(True)
    ax.set_aspect("equal", adjustable="box")

    plt.subplots_adjust(right=0.75)
    ax.legend(loc="upper left", bbox_to_anchor=(1.02, 1))

    ax.text(1.02, 0.5,
            "\n".join(text_lines),
            transform=ax.transAxes,
            fontsize=11, color="green",
            va="center", ha="left")

    out_path = os.path.join(FIG_ROOT, f"{folder_name}_location.png")
    plt.savefig(out_path, dpi=180, bbox_inches="tight")
    plt.close()
    print(f"[Saved] {out_path}")

# --------------------------------------------------------------
# Iterate folders
# --------------------------------------------------------------
# for name in ["Top_Unlab", "RuJun", "YuXiang", "Empty_Unlab", "BoYu", "YuanQi", "Jody"]:
for name in ["YuXiang_5mins_Active", "YuXiang_5mins_Passive",
             "Door_5mins_Active", "Door_5mins_Passive",
             "Bottom_5mins_Active", "Bottom_5mins_Passive", "Top_Unlab", "RuJun", "YuXiang", "Empty_Unlab", "BoYu", "YuanQi", "Jody"]:
    coords_path = os.path.join(PRED_ROOT, f"{name}_coords.npy")
    probs_path  = os.path.join(PRED_ROOT, f"{name}_probs.npy")

    if not os.path.exists(coords_path):
        print(f"⚠️ Missing {coords_path}, skip")
        continue

    coords = np.load(coords_path)
    probs  = np.load(probs_path) if os.path.exists(probs_path) else None
    aoa_ok_path = os.path.join(PRED_ROOT, f"{name}_aoa_ok.npy")
    if os.path.exists(aoa_ok_path):
        aoa_ok = np.load(aoa_ok_path).astype(bool)
        coords_ok = coords[aoa_ok]
        if coords_ok.shape[0] >= 20:
            print(f"[{name}] AoA-gated median XY:", np.median(coords_ok, axis=0),
                "  N_ok=", coords_ok.shape[0], "/", coords.shape[0])

    print(f"\nLoaded {coords.shape[0]} predictions for {name}")
    
    if os.path.exists(aoa_ok_path) and coords_ok.shape[0] >= 20:
        plot_coords(name, coords_ok, probs[aoa_ok] if probs is not None else None)
    else:
        plot_coords(name, coords, probs)


print("\n✔ Completed plotting + metrics for all unlabeled sets.")


[YuXiang_5mins_Active] AoA-gated median XY: [1.1435544 2.356823 ]   N_ok= 2048 / 2738

Loaded 2738 predictions for YuXiang_5mins_Active
[YuXiang_5mins_Active] Localization:
  p90_m: 3.161
  within_0.5m_%: 2.686
  within_1.0m_%: 7.373
  within_2.0m_%: 17.529
[YuXiang_5mins_Active] Presence:
  Accuracy=1.000  P=1.000  R=1.000  F1=1.000
  CM [[TN FP],[FN TP]]:
 [[   0    0]
 [   0 2048]]
[Saved] /home/tonyliao/Location_new_aoa_PDF/unlabel_figures__20260115_None/YuXiang_5mins_Active_location.png
[YuXiang_5mins_Passive] AoA-gated median XY: [1.038856 2.377345]   N_ok= 1871 / 2556

Loaded 2556 predictions for YuXiang_5mins_Passive
[YuXiang_5mins_Passive] Localization:
  p90_m: 3.212
  within_0.5m_%: 1.015
  within_1.0m_%: 3.421
  within_2.0m_%: 8.765
[YuXiang_5mins_Passive] Presence:
  Accuracy=1.000  P=1.000  R=1.000  F1=1.000
  CM [[TN FP],[FN TP]]:
 [[   0    0]
 [   0 1871]]
[Saved] /home/tonyliao/Location_new_aoa_PDF/unlabel_figures__20260115_None/YuXiang_5mins_Passive_location.png
[Doo

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()
#Restart the kernel to free memory
import IPython
app = IPython.get_ipython()
app.kernel.do_shutdown(True)  # True = restart, False = shutdown

{'status': 'ok', 'restart': True}

: 